In [1]:
import platform
print(f"{platform.architecture()[0]} compatible with CPLEX/GLPK/SCIP")

64bit compatible with CPLEX/GLPK/SCIP


In [2]:
import pandas as pd
import numpy as np
try:
    from pyomo.environ import *
    print(f"Pyomo Version : {pyomo.__version__}")
except IOError:
    raise ImportError("Pyomo not available")


Pyomo Version : 6.6.2


In [3]:
class SolverAttribute(type):
    pass

In [4]:
class VRPTWSolver(object,metaclass=type):
    def __init__(self,data:pd.DataFrame, vehicle_max_capacity:int=200, instance_name:str="VRPTW_C101"):
        self.data = data
        self.customers = [i for i in range(1,len(self.data))]
        print(f"Number of Customers :: {len(self.customers)}")
        self.vertex = [0] + self.customers
        self.arc = [(i,j) for i in self.vertex for j in self.vertex if i!=j]
        self.c = {}
        self.c = self.distance_matrix()
        self.vehicle_max_capacity = vehicle_max_capacity
        self.ready_time = self.data["READY_TIME"].to_list()
        self.due_date = self.data["DUE_DATE"].to_list()
        self.model = ConcreteModel(instance_name)
        self.model.arc = Set(initialize=self.arc)
        self.model.dist = Param(self.model.arc,initialize=self.c)
        self.model.max_cap = Param(initialize=self.vehicle_max_capacity)
        self.model.cust = Set(initialize=self.customers)
        self.model.x = Var(self.model.arc,within=Binary)
        self.model.u = Var(self.model.cust,within=Reals,bounds=self._u_bounds)
        self.model.time2 = Var(self.vertex,bounds=self._time2_bounds)
        self.model.depart = Var(self.vertex,within=NonNegativeReals)
        self.model.objective = Objective()
        self.status = None
        self.model.con1 = ConstraintList()
        self.model.con2 = ConstraintList()
        self.model.con3=ConstraintList()
        self.model.con4 = ConstraintList()
        self.model.con5 = ConstraintList()
        self.solver = None
      
    def _distance(self,i,j):
        self.cust1_x = self.data["XCOORD"].iloc[i]
        self.cust1_y = self.data["YCOORD"].iloc[i]
        self.cust2_x = self.data["XCOORD"].iloc[j]
        self.cust2_y = self.data["YCOORD"].iloc[j]
        return round(np.sqrt((self.cust1_x-self.cust2_x)**2+(self.cust1_y-self.cust2_y)**2),2)
    
    def distance_matrix(self):
        for i,j in self.arc:
            self.c[(i,j)] = self._distance(i,j)
        return self.c
    
    def _u_bounds(self,model):
        return (0,self.model.max_cap)
    
    def _objective_expr(self):
        return Objective(expr=sum(self.c[(i,j)]*self.model.x[(i,j)] for i,j in self.model.arc), sense=minimize)
    
    
    
    def _constraint1(self):
        for i in self.model.cust:
            lhs = sum(self.model.x[i,j] for j in self.vertex if i!=j)
            rhs=1
            self.model.con1.add(lhs==rhs)
            
    def _constraint2(self):
        for j in self.model.cust:
            lhs= sum(self.model.x[i,j] for i in self.vertex if i!=j)
            rhs=1
            self.model.con2.add(lhs==rhs)
            
    def _constraint3(self):
        for i,j in self.model.arc:
            if i != 0 and j!=0:
                expr =  self.model.u[i] + self.data["DEMAND"].iloc[j] - (1-self.model.x[i,j])*99999 <= self.model.u[j]
                self.model.con3.add(expr)
        
    def _time2_bounds(self,model,i):
         return (self.ready_time[i],self.due_date[i])   
        
    
    def _constraint4(self):
        for i in self.model.cust:
            lhs = self.model.time2[i] + self.data["SERVICE_TIME"].iloc[i]
            rhs = self.model.depart[i]
            self.model.con4.add(lhs <= rhs)
            
    def _constraint5(self):
        for i,j in self.model.arc:
            if i != j and j != 0:
                 lhs = self.model.time2[i] + self.data["SERVICE_TIME"].iloc[i] + self.c[i,j] - (1-self.model.x[i,j])*99999
                 rhs = self.model.time2[j]
                 self.model.con5.add(lhs <= rhs)
                    
    def _check_math_model(self,verbose=True):
        if verbose:
            print(self.model.objective.expr)
            print("\n\n")
            print("s.t")
            print(self.model.con1[1].expr)
            print("\n")
            print(self.model.con2[1].expr)
            print("\n")
            print(self.model.con3[1].expr)
            print("\n")
            print(self.model.con4[1].expr)
            print("\n")
            print(self.model.con5[1].expr)
    
    def solve(self):
        self.model.objective = self._objective_expr()
        self._constraint1()
        self._constraint2()
        self._constraint3()
        self._constraint4()
        self._constraint5()
        self._check_math_model()
        self.solver = SolverFactory("scip",executable="C:/Program Files/SCIPOptSuite 8.0.1/bin/scip.exe")
        self.solver.options["timelimit"] = 60 #in seconds
        self.solver.options["limit/gap"] = 0.1 # 10%
        self.results = self.solver.solve(self.model,tee=True)
        for i,j in self.model.arc:
             if value(self.model.x[i,j]) > 0:
                print(f"x[{i},{j}]=1")
        print(f"Distance Traveresed : {value(self.model.objective)}")
        

In [5]:
dt = pd.read_csv("D:/VRPdataset/C101_200.csv")
solver = VRPTWSolver(data=dt)
solver.solve()

Number of Customers :: 100
'pyomo.core.base.objective.ScalarObjective'>) on block VRPTW_C101 with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
18.68*x[0,1] + 20.62*x[0,2] + 16.12*x[0,3] + 18.11*x[0,4] + 15.13*x[0,5] + 19.0*x[0,6] + 16.0*x[0,7] + 18.11*x[0,8] + 20.1*x[0,9] + 16.76*x[0,10] + 19.65*x[0,11] + 38.08*x[0,12] + 30.81*x[0,13] + 39.36*x[0,14] + 36.06*x[0,15] + 40.31*x[0,16] + 33.3*x[0,17] + 35.36*x[0,18] + 39.05*x[0,19] + 10.0*x[0,20] + 10.2*x[0,21] + 12.17*x[0,22] + 13.0*x[0,23] + 15.0*x[0,24] + 15.13*x[0,25] + 15.81*x[0,26] + 17.12*x[0,27] + 17.72*x[0,28] + 20.0*x[0,29] + 20.62*x[0,30] + 33.54*x[0,31] + 31.62*x[0,32] + 33.53*x[0,33] + 32.39*x[0,34] + 38.08*x[0,35] + 35.36*x[0,36] + 39.29*x[0,37] + 41.23*x[0,38] + 40.31*x[0,39] + 20.62*x[0,40] + 18.68*x[0,41] + 19.31*x[0,42] + 16.55*x[0,43] + 21.54*x[0,44] + 22.36*x[0,45] +

SCIP version 8.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 6.0.1] [GitHash: c84ee4283e]
Copyright (C) 2002-2022 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External libraries: 
  Soplex 6.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 8b86b300]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.5.2          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.1.0         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) [GitHash: 9363218]
  bliss 0.77           Computing Graph Automorphism Groups by T. Junttila and P. Kaski (www.tcs.hut.fi/Software/bliss/)
  Ipopt 

#### Using Decorator 

In [6]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from pyomo.opt import SolverStatus,TerminationCondition
import logging
from pyomo.util.infeasible import log_infeasible_constraints

In [7]:
class VRPTWDeco(object,metaclass=type):
    
    def __init__(self,data:pd.DataFrame, vehicle_max_capacity:int=200, instance_name:str="VRPTW_C101"):
        
        self.model = ConcreteModel(instance_name)
        
        self.model.data = data
        
        self.customers = [i for i in range(1,len(self.model.data))]
        print(f"Number of Customers :: {len(self.customers)}")
        
        
        self.model.vertex = [0] + self.customers
        self.arc = [(i,j) for i in self.model.vertex for j in self.model.vertex if i!=j]
        
        self.model.c = {}
        self.model.c = self.distance_matrix()
        self.vehicle_max_capacity = vehicle_max_capacity
        self.model.ready_time = self.model.data["READY_TIME"].to_list()
        self.model.due_date = self.model.data["DUE_DATE"].to_list()
        
        self.model.arc = Set(initialize=self.arc)
        self.model.dist = Param(self.model.arc,initialize=self.model.c)
        self.model.max_cap = Param(initialize=self.vehicle_max_capacity)
        self.model.cust = Set(initialize=self.customers)
        self.model.x = Var(self.model.arc,within=Binary)
        self.model.u = Var(self.model.cust,within=Reals,bounds=self._u_bounds)
        self.model.time2 = Var(self.model.vertex,bounds=self._time2_bounds)
        self.model.depart = Var(self.model.vertex,within=NonNegativeReals)
        #self.model.objective = Objective()
        self.model.status = None
        #self.solver = None
        
        #You need to write objective function and constraints within init() otherwise it will give error "self" not defined
        '''Objective Function'''
        @self.model.Objective()
        def obj(s):
            return sum(s.c[(i,j)]*s.x[(i,j)] for i,j in s.arc)
        
        '''Constraint 1'''
        @self.model.Constraint(self.model.cust)
        def _constraint1(s,i):
            lhs = sum(s.x[i,j] for j in s.vertex if i!=j)
            rhs=1
            return lhs==rhs
        
        '''Constraint 2'''
        @self.model.Constraint(self.model.cust)
        def _constraint2(s,j):
            lhs= sum(s.x[i,j] for i in s.vertex if i!=j)
            rhs=1
            return lhs==rhs
        
        '''constraint 3'''
        @self.model.Constraint(self.model.arc)
        def _constraint3(s,i,j):
                if i != 0 and j!=0:
                     return  s.u[i] + s.data["DEMAND"].iloc[j] - (1-s.x[i,j])*99999 <= s.u[j]
                else:
                     return Constraint.Skip
                     
            
        
        '''Constraint 4'''
        @self.model.Constraint(self.model.cust)
        def _constraint4(s,i):
            lhs = s.time2[i] + s.data["SERVICE_TIME"].iloc[i]
            rhs = s.depart[i]
            return lhs <= rhs
        
        '''Constraint 5'''
        @self.model.Constraint(self.model.arc)
        def _constraint5(s,i,j):
            if i != j and j != 0:
                 lhs = s.time2[i] + s.data["SERVICE_TIME"].iloc[i] + s.c[i,j] - (1-s.x[i,j])*99999
                 rhs = s.time2[j]
                 return lhs <= rhs
            else:
                return Constraint.Skip
      
    def _distance(self,i,j):
        self.cust1_x = self.model.data["XCOORD"].iloc[i]
        self.cust1_y = self.model.data["YCOORD"].iloc[i]
        self.cust2_x = self.model.data["XCOORD"].iloc[j]
        self.cust2_y = self.model.data["YCOORD"].iloc[j]
        return round(np.sqrt((self.cust1_x-self.cust2_x)**2+(self.cust1_y-self.cust2_y)**2),2)
    
    def distance_matrix(self):
        for i,j in self.arc:
            self.model.c[(i,j)] = self._distance(i,j)
        return self.model.c
    
    def _u_bounds(self,model):
        return (0,self.model.max_cap)
    
     
    def _time2_bounds(self,model,i):
         return (self.model.ready_time[i],self.model.due_date[i])   
        
                        
    def _check_math_model(self,verbose=True):
        if verbose:
            print(self.model.objective.expr)
            print("\n\n")
            print("s.t")
            print(self.model.con1[1].expr)
            print("\n")
            print(self.model.con2[1].expr)
            print("\n")
            print(self.model.con3[1].expr)
            print("\n")
            print(self.model.con4[1].expr)
            print("\n")
            print(self.model.con5[1].expr)
    
    def solve(self):
        self.solver = SolverFactory("scip",executable="C:/Program Files/SCIPOptSuite 8.0.1/bin/scip.exe")
        self.solver.options["timelimit"] = 60 #in seconds
        self.solver.options["limit/gap"] = 0.1 # 10%
        self.results = self.solver.solve(self.model,tee=True)
        
        '''Write LP File'''
        self.model.write("D:/VRPTW_DECORATOR.lp")
        
        if (self.results.solver.status == SolverStatus.ok) and (self.results.solver.termination_condition == TerminationCondition.optimal):
            print("\n\n Optimal Solution")
        elif (self.results.solver.termination_condition == TerminationCondition_infeasible):
            '''Check for Infeasibility'''
            print("\n\n Infeasible Solution")
            log_infeasible_constraints(self.model, log_expression =True, log_variables=True)
            logging.basicConfig(filename="D:/VRPTW_infeasibility.log",encoding ='utf-8',level = logging.INFO)
        else:
            print(f"\n\n Solver Status : {self.result.solver.status}")
        for i,j in self.model.arc:
             if value(self.model.x[i,j]) > 0:
                print(f"x[{i},{j}]=1")
        print(f"Distance Traveresed : {value(self.model.obj())}")
        
    
        
        

In [8]:
dt = pd.read_csv("D:/VRPdataset/C101_200.csv")
vrptw = VRPTWDeco(data=dt)
vrptw.solve()


Number of Customers :: 100
SCIP version 8.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 6.0.1] [GitHash: c84ee4283e]
Copyright (C) 2002-2022 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

External libraries: 
  Soplex 6.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 8b86b300]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.5.2          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.1.0         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) [GitHash: 9363218]
  bliss 0.77           Computing Graph Automorphism Groups by T. Junttila and P. Kaski (www.tcs.hut.f

In [15]:
print(f"Distance Traveresed : {value(vrptw.model.obj())}")

Distance Traveresed : 829.0100000000001


In [1]:
$ ipython locate

SyntaxError: invalid syntax (<ipython-input-1-d870970cb45f>, line 1)